In [27]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import time 

import torch
import mrmr
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [28]:
# Constants
DATA_PATH = './data/'
DATASET_PATH = DATA_PATH + 'smartphone+based+recognition+of+human+activities+and+postural+transitions/'
MODELS_PATH = DATA_PATH + 'models/raw-models/'

In [29]:
# Load the data
train_set = pd.read_csv(DATA_PATH + "self-calculated/raw-data/train-data.txt", sep='\s+', header=None)
test_set = pd.read_csv(DATA_PATH + "self-calculated/raw-data/test-data.txt", sep='\s+', header=None)

train_set.columns=['accX', 'accY', 'accZ', 'gyroX', 'gyroY', 'gyroZ', "subject", "activity"]
test_set.columns=['accX', 'accY', 'accZ', 'gyroX', 'gyroY', 'gyroZ', "subject", "activity"]

print(train_set.shape)
print(test_set.shape)

(570929, 8)
(244685, 8)


In [30]:
# Creating model
device = None

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

class RawDataModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(7, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.05),
            nn.Linear(128, 32),
            nn.SiLU(),
            nn.Linear(32, 8),
            nn.Softmax()  # or nn.Softmax(dim=1) for multi-class classification
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
    
model = RawDataModel().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [31]:
# Training and testing function

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        y = y.squeeze()
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            y = y.squeeze()
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    return(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [32]:
# Dataset preparation
class HAPTDataset(Dataset):
    def __init__(self, dataset, features, label):
        self.data = torch.tensor(dataset[features].values, dtype=torch.float32)
        self.labels = torch.tensor(dataset[label].values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
train_dataset = HAPTDataset(train_set, train_set.columns[:-1], 'activity')
test_dataset = HAPTDataset(test_set, test_set.columns[:-1], 'activity')

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)

In [33]:
epochs = 5
perf_timer = time.perf_counter()
perf_acc = ""

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_function, optimizer)
    perf_acc = test(test_dataloader, model, loss_function)
    
print(perf_acc)
perf_timer = time.perf_counter() - perf_timer
print("Done!")

Epoch 1
-------------------------------
loss: 2.082390  [    8/570929]
loss: 1.736721  [  808/570929]
loss: 1.756289  [ 1608/570929]
loss: 2.015700  [ 2408/570929]
loss: 1.994574  [ 3208/570929]
loss: 1.740266  [ 4008/570929]
loss: 1.823089  [ 4808/570929]
loss: 2.069905  [ 5608/570929]
loss: 1.969811  [ 6408/570929]
loss: 1.915291  [ 7208/570929]
loss: 1.803608  [ 8008/570929]
loss: 1.784012  [ 8808/570929]
loss: 1.633013  [ 9608/570929]
loss: 1.759413  [10408/570929]
loss: 1.506513  [11208/570929]
loss: 1.979693  [12008/570929]
loss: 1.612426  [12808/570929]
loss: 1.426604  [13608/570929]
loss: 1.987725  [14408/570929]
loss: 1.549052  [15208/570929]
loss: 1.817516  [16008/570929]
loss: 1.684618  [16808/570929]
loss: 1.598332  [17608/570929]
loss: 1.726735  [18408/570929]
loss: 1.697989  [19208/570929]
loss: 1.328940  [20008/570929]
loss: 1.733185  [20808/570929]
loss: 1.413156  [21608/570929]
loss: 1.701856  [22408/570929]
loss: 2.052372  [23208/570929]
loss: 1.800381  [24008/570929]

ValueError: Expected input batch_size (1) to match target batch_size (0).